# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
print(os.getcwd())
filepath = os.getcwd() + '/event_data'
for root, dirs, files in os.walk(filepath):    
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
full_data_rows_list = [] 
for f in file_path_list:

    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        for line in csvreader:
            full_data_rows_list.append(line) 
            
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [ ]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




song_sessions is partitioned by session_id and item_in_session, since the query will filter by those columns, in respective order (session_id first, then item_in_session). No clustering column needed.

In [14]:
query = """
  DROP TABLE IF EXISTS song_sessions
"""
try:
    session.execute(query)
except Exception as e:
    print(e)

query = """
  CREATE TABLE IF NOT EXISTS song_sessions
  (
    session_id int, item_in_session int, artist text, song text, length float,
    PRIMARY KEY (session_id, item_in_session)
  )
"""
try:
    session.execute(query)
except Exception as e:
    print(e)

                    

In [15]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_sessions (session_id, item_in_session, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [19]:
query1 = """
  SELECT artist, song, length FROM song_sessions WHERE session_id = 338
    AND item_in_session = 4
"""
try:
    rows = session.execute(query1)

    for row in rows:
        print (row.artist, row.song, row.length)
except Exception as e:
    print(e)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

song_users is partitioned by combination of user_id AND session id, since the query will filter by those columns. Additional clustering column needed for sorting, which is item_in_session.

In [20]:
query = """
  DROP TABLE IF EXISTS song_users
"""
try:
    session.execute(query)
except Exception as e:
    print(e)

query = """
  CREATE TABLE IF NOT EXISTS song_users
  (
    user_id int, session_id int, item_in_session int, artist text, 
    song text, user_first_name text, user_last_name text, 
    PRIMARY KEY ( (user_id, session_id), item_in_session)
  )
"""
try:
    session.execute(query)
except Exception as e:
    print(e)

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_users (user_id, session_id, item_in_session, artist, song, user_first_name, user_last_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
        
query2 = """
  SELECT artist, song, user_first_name, user_last_name FROM song_users WHERE user_id = 10 AND session_id = 182
"""
try:
    rows = session.execute(query2)

    for row in rows:
        print (row.artist, row.song, row.user_first_name, row.user_last_name)
except Exception as e:
    print(e)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


user_listens is partitioned by combination of song and user_id, since the query will filter by those columns. Primary key is combination, to make sure that each user that listens to particular song entered (not just last user-song combination in dataset).
No clustering columnn needed

In [22]:
query = """
  DROP TABLE IF EXISTS user_listens
"""
try:
    session.execute(query)
except Exception as e:
    print(e)

query = """
  CREATE TABLE IF NOT EXISTS user_listens
  (
    song text, user_id int, user_first_name text, user_last_name text,
    PRIMARY KEY (song, user_id)
  )
"""
try:
    session.execute(query)
except Exception as e:
    print(e)

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_listens (song, user_id, user_first_name, user_last_name)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))
        
query3 = """
  SELECT user_first_name, user_last_name FROM user_listens WHERE song = 'All Hands Against His Own'
"""
try:
    rows = session.execute(query3)

    for row in rows:
        print (row.user_first_name, row.user_last_name)
except Exception as e:
    print(e)
                    

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [21]:
try:
    query = "DROP TABLE IF EXISTS song_sessions"
    session.execute(query)

    query = "DROP TABLE IF EXISTS song_users"
    session.execute(query)

    query = "DROP TABLE IF EXISTS user_listens"
    session.execute(query)
except Exception as e:
    print(e)


### Close the session and cluster connection¶

In [22]:
session.shutdown()
cluster.shutdown()